### From a URL create a Dataframe

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
from tqdm.notebook import tqdm
import random

In [2]:
url = "https://www.billboard.com/charts/hot-100/"

In [3]:
top100 = requests.get(url)

In [4]:
top100.status_code

200

### Search for the songs in the URL

In [5]:
soup=BeautifulSoup(top100.content,'html.parser')

In [6]:
soup.select('h3.c-title.a-no-trucate')[0].text

'\nEasy On Me\n'

In [7]:
top100=len(soup.select('h3.c-title.a-no-trucate'))

In [8]:
top100

100

In [9]:
index = soup.select('h3.c-title.a-no-trucate')[0].get_text(strip=True)

In [10]:
index

'Easy On Me'

### Search for the artists in the URL

In [11]:
soup.select('span.c-label.a-no-trucate')[0].get_text(strip=True)

'Adele'

In [12]:
soup.select('span.c-label.a-no-trucate')[99].get_text(strip=True)

'The Weeknd'

### Create a DataFrame

In [13]:
title = []
artist = []

In [14]:
for i in tqdm(range(top100)):
    title.append(soup.select('h3.c-title.a-no-trucate')[i].get_text(strip=True))
    artist.append(soup.select('span.c-label.a-no-trucate')[i].get_text(strip=True))

  0%|          | 0/100 [00:00<?, ?it/s]

In [15]:
songs_top100=pd.DataFrame({'Title':title,'Artist':artist})

In [16]:
songs_top100

,Title,Artist
0,Easy On Me,Adele
1,Stay,The Kid LAROI & Justin Bieber
2,Industry Baby,Lil Nas X & Jack Harlow
3,All Too Well (Taylor's Version),Taylor Swift
4,Oh My God,Adele
...,...,...
95,Too Easy,Gunna & Future
96,Escape Plan,Travis Scott
97,Who's In Your Head,Jonas Brothers
98,Jugaste y Sufri,Eslabon Armado Featuring DannyLux


In [17]:
songs_top100[92:]

,Title,Artist
92,Volvi,Aventura x Bad Bunny
93,Maybach,42 Dugg Featuring Future
94,Baddest,"Yung Bleu, Chris Brown & 2 Chainz"
95,Too Easy,Gunna & Future
96,Escape Plan,Travis Scott
97,Who's In Your Head,Jonas Brothers
98,Jugaste y Sufri,Eslabon Armado Featuring DannyLux
99,Take My Breath,The Weeknd


### Create a prototype where:
- The user can ask for one song and see if its in the playlist Top hot 100 songs
- Returns a new song if the first one is in the playlist

In [18]:
song= input("Give us a song name to get a recommendation!: ")

Give us a song name to get a recommendation!: stay


In [19]:
check = songs_top100[songs_top100['Title'].str.lower().str.replace(" "," ").str.contains(song)]

In [20]:
index = check.index.tolist()

In [21]:
def recommendation_engine():
    recommendation_engine()

In [22]:
song= input("Give us a song name to get a recommendation!: ")

check = songs_top100[songs_top100['Title'].str.lower().str.replace(" "," ").str.contains(song)]
index = check.index.tolist()

if len(index) == 0:
    print("This song is not on the Top 100 hot playlist! ")
else:
    answer = input("Do you mean " + songs_top100.Title[index].values[0] + " by " + songs_top100.Artist[index].values[0] + "?")

    if answer.lower() == 'yes':
        suggestion = songs_top100.sample().index.tolist()
        print("Nice! This is a hot song! You might also like " + songs_top100['Title'][suggestion].item() + " by " + songs_top100['Artist'][suggestion].item())

    else:
        print("that song is not in the playlist, try another song!")
    
    

Give us a song name to get a recommendation!: stay
Do you mean Stay by The Kid LAROI & Justin Bieber?yes
Nice! This is a hot song! You might also like God's Country by State Of Mine & Drew Jacobs


# Import Spotify

In [23]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

### Credentials

In [24]:
import getpass

client_id=str(getpass.getpass('client_id?'))
client_secret=str(getpass.getpass('client_secret?'))

client_id?········
client_secret?········


In [25]:
# embedding our getpass credentials in the access key

sp= spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

## Import playlists by genre
- I will first import a POP playlist

In [26]:
## https://open.spotify.com/playlist/37i9dQZF1DX1ngEVM0lKrb?si=332a03edd1db4135
## 37i9dQZF1DX1ngEVM0lKrb

In [27]:
pop_playlist = sp.user_playlist_tracks("Sarah Eringaard", "1HWoagBCUTlqigP88enL5r")

In [29]:
pop_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [31]:
len(pop_playlist['items'])

100

In [32]:
def get_playlist_tracks(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [33]:
results2=get_playlist_tracks("Sarah Eringaard", "1HWoagBCUTlqigP88enL5r")

In [34]:
len(results2)

473

In [35]:
results2[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [36]:
results2[0]['track']["uri"].rsplit(':',1)

['spotify:track', '04aAxqtGp5pv12UXAg4pkq']

In [37]:
results2[0]['track']["artists"][0].keys()

dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])

In [38]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results2

In [39]:
for r in results2:
    song_URI.append(r["track"]["uri"])

In [40]:
for r in results2: 
    song_ID.append(r["track"]["id"])

In [41]:
for r in results2: 
    song_name.append(r["track"]["name"])

In [42]:
for r in results2: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [43]:
import pandas as pd

In [44]:
playlist_pop_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})


In [45]:
playlist_pop_info

,song_name,song_ID,song_artist
0,Centuries,04aAxqtGp5pv12UXAg4pkq,Fall Out Boy
1,Counting Stars,6sy3LkhNFjJWlaeSMNwQ62,OneRepublic
2,Animals,16pwlVsypm4aDKMhXdOuXg,Maroon 5
3,Take Me to Church,0aOluBqXYd0rFSCsgDyAWX,Hozier
4,Rude,6RtPijgfPKROxEzTHNRiDp,MAGIC!
...,...,...,...
468,Jenny,2rpVmxAGGc18wpBREaoE0R,Studio Killers
469,I'm Gonna Be (500 Miles),66S14BkJDxgkYxLl5DCqOz,The Proclaimers
470,My Humps,19JdbdEwQI7bCxefMokZO8,Black Eyed Peas
471,I Just Wanna Run,3WLLJzllBEgPiSA0qd3BN3,The Downtown Fiction


### Add a new column for genre in playlist_pop_info

In [46]:
playlist_pop_info.insert(3, 'Genre', 'Pop') 

In [47]:
playlist_pop_info

,song_name,song_ID,song_artist,Genre
0,Centuries,04aAxqtGp5pv12UXAg4pkq,Fall Out Boy,Pop
1,Counting Stars,6sy3LkhNFjJWlaeSMNwQ62,OneRepublic,Pop
2,Animals,16pwlVsypm4aDKMhXdOuXg,Maroon 5,Pop
3,Take Me to Church,0aOluBqXYd0rFSCsgDyAWX,Hozier,Pop
4,Rude,6RtPijgfPKROxEzTHNRiDp,MAGIC!,Pop
...,...,...,...,...
468,Jenny,2rpVmxAGGc18wpBREaoE0R,Studio Killers,Pop
469,I'm Gonna Be (500 Miles),66S14BkJDxgkYxLl5DCqOz,The Proclaimers,Pop
470,My Humps,19JdbdEwQI7bCxefMokZO8,Black Eyed Peas,Pop
471,I Just Wanna Run,3WLLJzllBEgPiSA0qd3BN3,The Downtown Fiction,Pop


## Import playlists by genre
- I will import now a INDIE playlist

In [48]:
## https://open.spotify.com/playlist/7lnCgcTxLTTcOqvgoS80sC?si=923d97089cf041ea
## oceanify

In [49]:
indie_playlist = sp.user_playlist_tracks("oceanify", "7lnCgcTxLTTcOqvgoS80sC")

In [51]:
indie_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [53]:
len(indie_playlist['items'])

100

In [54]:
def get_playlist_tracks1(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [55]:
results3=get_playlist_tracks1("oceanify", "7lnCgcTxLTTcOqvgoS80sC")

In [56]:
len(results3)

166

In [57]:
results3[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [58]:
results3[0]['track']["uri"].rsplit(':',1)

['spotify:track', '3Wrjm47oTz2sjIgck11l5e']

In [59]:
results3[0]['track']["artists"][0].keys()

dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])

In [60]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results3

In [61]:
for r in results3:
    song_URI.append(r["track"]["uri"])
    
for r in results3: 
    song_ID.append(r["track"]["id"])
    
for r in results3: 
    song_name.append(r["track"]["name"])
    
for r in results3: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [62]:
playlist_indie_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})


In [63]:
playlist_indie_info

,song_name,song_ID,song_artist
0,Beggin',3Wrjm47oTz2sjIgck11l5e,Måneskin
1,The Less I Know The Better,4g3Ax56IslQkI6XVfYKVc5,Tame Impala
2,Mr. Brightside,3n3Ppam7vgaVa1iaRUc9Lp,The Killers
3,Boys Don't Cry,1QFh8OH1e78dGd3VyJZCAC,The Cure
4,A-Punk,5dKBaysNJtfpyNTRa5lqDb,Vampire Weekend
...,...,...,...
161,Maybe - Extended Version,5M9FFdvLfKPkORIRERHzKY,Elizabeth Cuite
162,Deja Vu,5PWT6FhFPn2E5dv5tUuqNE,Luvlee
163,One in a Million,0B2gr2o765XiWU6dGKmktU,Will Whisson
164,Comin’ Home (Stewart Sullivan Mix),49FulGQwWUrWVVB69TfHAr,Dominoe


### Add a new column for genre in playlist_indie_info

In [64]:
playlist_indie_info.insert(3, 'Genre', 'Indie') 

In [65]:
playlist_indie_info

,song_name,song_ID,song_artist,Genre
0,Beggin',3Wrjm47oTz2sjIgck11l5e,Måneskin,Indie
1,The Less I Know The Better,4g3Ax56IslQkI6XVfYKVc5,Tame Impala,Indie
2,Mr. Brightside,3n3Ppam7vgaVa1iaRUc9Lp,The Killers,Indie
3,Boys Don't Cry,1QFh8OH1e78dGd3VyJZCAC,The Cure,Indie
4,A-Punk,5dKBaysNJtfpyNTRa5lqDb,Vampire Weekend,Indie
...,...,...,...,...
161,Maybe - Extended Version,5M9FFdvLfKPkORIRERHzKY,Elizabeth Cuite,Indie
162,Deja Vu,5PWT6FhFPn2E5dv5tUuqNE,Luvlee,Indie
163,One in a Million,0B2gr2o765XiWU6dGKmktU,Will Whisson,Indie
164,Comin’ Home (Stewart Sullivan Mix),49FulGQwWUrWVVB69TfHAr,Dominoe,Indie


## Import playlists by genre
- I will import now a ROCK playlist

In [66]:
## https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=6870797da01948f7
## Spotify

In [67]:
rock_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DWXRqgorJj26U")

In [69]:
rock_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [71]:
len(rock_playlist['items'])

100

In [72]:
def get_playlist_tracks2(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [73]:
results4=get_playlist_tracks2("Spotify", "37i9dQZF1DWXRqgorJj26U")

In [74]:
len(results4)

166

In [75]:
results4[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [76]:
results4[0]['track']["uri"].rsplit(':',1)

['spotify:track', '57JVGBtBLCfHw2muk5416J']

In [77]:
results4[0]['track']["artists"][0].keys()

dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])

In [78]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results4

In [79]:
for r in results4:
    song_URI.append(r["track"]["uri"])
    
for r in results4: 
    song_ID.append(r["track"]["id"])
    
for r in results4: 
    song_name.append(r["track"]["name"])
    
for r in results4: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [80]:
playlist_rock_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})

In [81]:
playlist_rock_info

,song_name,song_ID,song_artist
0,Another One Bites The Dust - Remastered 2011,57JVGBtBLCfHw2muk5416J,Queen
1,Paradise City,3YBZIN3rekqsKxbJc9FZko,Guns N' Roses
2,Black Dog - Remaster,3qT4bUD1MaWpGrTwcvguhb,Led Zeppelin
3,Highway to Hell,2zYzyRzz6pRmhPzyfMEC8s,AC/DC
4,Dream On,5MxNLUsfh7uzROypsoO5qe,Aerosmith
...,...,...,...
161,Ramble On - 1990 Remaster,3MODES4TNtygekLl146Dxd,Led Zeppelin
162,Crazy On You,5zH710lFSLtkHbMkslLDjR,Heart
163,Foreplay / Long Time,5UYPZZDQfpCE7A0FDDqv4k,Boston
164,My Generation - Mono Version,45s88Xopo6KvHc0PQ05aGg,The Who


### Add a new column for genre in playlist_rock_info

In [82]:
playlist_rock_info.insert(3, 'Genre', 'Rock') 

In [83]:
playlist_rock_info

,song_name,song_ID,song_artist,Genre
0,Another One Bites The Dust - Remastered 2011,57JVGBtBLCfHw2muk5416J,Queen,Rock
1,Paradise City,3YBZIN3rekqsKxbJc9FZko,Guns N' Roses,Rock
2,Black Dog - Remaster,3qT4bUD1MaWpGrTwcvguhb,Led Zeppelin,Rock
3,Highway to Hell,2zYzyRzz6pRmhPzyfMEC8s,AC/DC,Rock
4,Dream On,5MxNLUsfh7uzROypsoO5qe,Aerosmith,Rock
...,...,...,...,...
161,Ramble On - 1990 Remaster,3MODES4TNtygekLl146Dxd,Led Zeppelin,Rock
162,Crazy On You,5zH710lFSLtkHbMkslLDjR,Heart,Rock
163,Foreplay / Long Time,5UYPZZDQfpCE7A0FDDqv4k,Boston,Rock
164,My Generation - Mono Version,45s88Xopo6KvHc0PQ05aGg,The Who,Rock


## Import playlists by genre
- I will import now a TROPICAL HOUSE playlist

In [84]:
## https://open.spotify.com/playlist/37i9dQZF1DX0AMssoUKCz7?si=8509345b0f7b4117
## Spotify

In [85]:
tropical_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX0AMssoUKCz7")

In [87]:
tropical_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [89]:
len(tropical_playlist['items'])

100

In [90]:
def get_playlist_tracks3(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [91]:
results5=get_playlist_tracks3("Spotify", "37i9dQZF1DX0AMssoUKCz7")

In [92]:
len(results5)

200

In [93]:
results5[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [94]:
results5[0]['track']["uri"].rsplit(':',1)

['spotify:track', '1c1sdxrYLIiuJOlE7PPttb']

In [95]:
results5[0]['track']["artists"][0].keys()

dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])

In [96]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results5

In [97]:
for r in results5:
    song_URI.append(r["track"]["uri"])
    
for r in results5: 
    song_ID.append(r["track"]["id"])
    
for r in results5: 
    song_name.append(r["track"]["name"])
    
for r in results5: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [98]:
playlist_tropical_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})

In [99]:
playlist_tropical_info

,song_name,song_ID,song_artist
0,Love Me Now (feat. Zoe Wees),1c1sdxrYLIiuJOlE7PPttb,Kygo
1,Skinny Dip (Komodo) [feat. Philip Strand],79m1I109FyZMx3VwQZAEQM,Deepend
2,Summer Feeling,793OuR8rJ0zpVp708ONAxO,Matoma
3,Ride,4C020pCjavabKAGmjZZRmh,Madism
4,Call On Me (feat. Georgia Ku),3pBv3LfvfPft2TqHleqKHo,Sam Feldt
...,...,...,...
195,Sunset,5jUgFofZJCuSQjcYao0VMv,A-SHO
196,Riptide - FlicFlac Remix,5K9AcgSi8DK4lZKM2LpFu8,Vance Joy
197,High And Low,4PwXNVqkIM8KBuEgYFqDiE,Sam Feldt
198,Beat the Sunrise,7th2B8tA26On1RAt2WmnMI,SNBRN


### Add a new column for genre in playlist_tropical_info

In [100]:
playlist_tropical_info.insert(3, 'Genre', 'Tropical') 

In [101]:
playlist_tropical_info

,song_name,song_ID,song_artist,Genre
0,Love Me Now (feat. Zoe Wees),1c1sdxrYLIiuJOlE7PPttb,Kygo,Tropical
1,Skinny Dip (Komodo) [feat. Philip Strand],79m1I109FyZMx3VwQZAEQM,Deepend,Tropical
2,Summer Feeling,793OuR8rJ0zpVp708ONAxO,Matoma,Tropical
3,Ride,4C020pCjavabKAGmjZZRmh,Madism,Tropical
4,Call On Me (feat. Georgia Ku),3pBv3LfvfPft2TqHleqKHo,Sam Feldt,Tropical
...,...,...,...,...
195,Sunset,5jUgFofZJCuSQjcYao0VMv,A-SHO,Tropical
196,Riptide - FlicFlac Remix,5K9AcgSi8DK4lZKM2LpFu8,Vance Joy,Tropical
197,High And Low,4PwXNVqkIM8KBuEgYFqDiE,Sam Feldt,Tropical
198,Beat the Sunrise,7th2B8tA26On1RAt2WmnMI,SNBRN,Tropical


## Import playlists by genre
- I will import now a HIPHOP playlist

In [102]:
## https://open.spotify.com/playlist/33I6RpefRQcRh69xEczaKT?si=8bc85c31fb894e45
## Leon Bouw

In [103]:
hiphop_playlist = sp.user_playlist_tracks("Leon Bouw", "33I6RpefRQcRh69xEczaKT")

In [105]:
hiphop_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [107]:
len(hiphop_playlist['items'])

100

In [108]:
def get_playlist_tracks4(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [109]:
results6=get_playlist_tracks3("Leon Bouw", "33I6RpefRQcRh69xEczaKT")

In [110]:
len(results6)

265

In [111]:
results6[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [112]:
results6[0]['track']["uri"].rsplit(':',1)

['spotify:track', '1YrHyDbH17btvI2rFTkbCQ']

In [113]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results6

In [114]:
for r in results6:
    song_URI.append(r["track"]["uri"])
    
for r in results6: 
    song_ID.append(r["track"]["id"])
    
for r in results6: 
    song_name.append(r["track"]["name"])
    
for r in results6: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [115]:
playlist_hiphop_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})

In [116]:
playlist_hiphop_info

,song_name,song_ID,song_artist
0,Bitch Please,1YrHyDbH17btvI2rFTkbCQ,Snoop Dogg
1,How We Do,6mZcNu9i8uhOYMiEAyEx81,The Game
2,No Hands (feat. Roscoe Dash & Wale),6GgPsuz0HEO0nrO2T0QhDv,Waka Flocka Flame
3,Gorgeous,1yhQGEykn7ZTNLwcIjOAwZ,Kanye West
4,99 Problems,4HsL5A46wa0O2jQ0ngj3H7,JAY-Z
...,...,...,...
260,Yo Sassy Ways,0J1siBLQBHYgYKeFgu7RAY,Warren G
261,Cisco Kid,0CJ3Y3V6UQn5X7IjXDO3IO,Redman
262,Woah! - Radio Mix,5fZjSIReYPJIPwpvrsyW3o,Black Rob
263,I Don't Wanna Care Right Now (feat. MDMA),5yhXLtQL9waNMjgkl8MsMP,Lupe Fiasco


### Add a new column for genre in playlist_hiphop_info

In [117]:
playlist_hiphop_info.insert(3, 'Genre', 'HipHop') 

In [118]:
playlist_hiphop_info

,song_name,song_ID,song_artist,Genre
0,Bitch Please,1YrHyDbH17btvI2rFTkbCQ,Snoop Dogg,HipHop
1,How We Do,6mZcNu9i8uhOYMiEAyEx81,The Game,HipHop
2,No Hands (feat. Roscoe Dash & Wale),6GgPsuz0HEO0nrO2T0QhDv,Waka Flocka Flame,HipHop
3,Gorgeous,1yhQGEykn7ZTNLwcIjOAwZ,Kanye West,HipHop
4,99 Problems,4HsL5A46wa0O2jQ0ngj3H7,JAY-Z,HipHop
...,...,...,...,...
260,Yo Sassy Ways,0J1siBLQBHYgYKeFgu7RAY,Warren G,HipHop
261,Cisco Kid,0CJ3Y3V6UQn5X7IjXDO3IO,Redman,HipHop
262,Woah! - Radio Mix,5fZjSIReYPJIPwpvrsyW3o,Black Rob,HipHop
263,I Don't Wanna Care Right Now (feat. MDMA),5yhXLtQL9waNMjgkl8MsMP,Lupe Fiasco,HipHop


## Import playlists by genre
- I will import now a LATIN playlist

In [119]:
## https://open.spotify.com/playlist/4DLmZ0UM8TH0UXRfI0Mvu6?si=138328e1e1b44091
## MiKeL

In [120]:
latin_playlist = sp.user_playlist_tracks("MiKeL", "4DLmZ0UM8TH0UXRfI0Mvu6")

In [122]:
latin_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [124]:
len(latin_playlist['items'])

100

In [125]:
def get_playlist_tracks5(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [126]:
results7=get_playlist_tracks5("MiKeL", "4DLmZ0UM8TH0UXRfI0Mvu6")

In [127]:
len(results7)

269

In [128]:
results7[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [129]:
results7[0]['track']["uri"].rsplit(':',1)

['spotify:track', '1ZcrHpBbILPNfhBtPhQVHh']

In [130]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results7

In [131]:
for r in results7:
    song_URI.append(r["track"]["uri"])
    
for r in results7: 
    song_ID.append(r["track"]["id"])
    
for r in results7: 
    song_name.append(r["track"]["name"])
    
for r in results7: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [132]:
playlist_latin_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})

In [133]:
playlist_latin_info

,song_name,song_ID,song_artist
0,Se Menea,1ZcrHpBbILPNfhBtPhQVHh,Don Omar
1,Una Vaina Loca,2qEoAz0i6yEz5dPggABcLH,Fuego
2,Señor Juez,2PH533J1xudlfZ5GaHAdVX,Ozuna
3,Ella - Remix,7iHHlCbsFou5DZSW5HCPfs,Boza
4,Unfollow,6GsD646HrIDJJpzoY9T1Rt,Duki
...,...,...,...
264,Ay Vamos,6Ges5C2IE738iJh4HyQizQ,J Balvin
265,Love Not War (The Tampa Beat),4Lt6GXGzYsa1tgkv3nGSTm,Jason Derulo
266,BICHOTA,7vrJn5hDSXRmdXoR30KgF1,KAROL G
267,Hey DJ,1XgpK29CGGjZnxPYkiRbh4,CNCO


### Add a new column for genre in playlist_latin_info

In [134]:
playlist_latin_info.insert(3, 'Genre', 'Latin') 

In [135]:
playlist_latin_info

,song_name,song_ID,song_artist,Genre
0,Se Menea,1ZcrHpBbILPNfhBtPhQVHh,Don Omar,Latin
1,Una Vaina Loca,2qEoAz0i6yEz5dPggABcLH,Fuego,Latin
2,Señor Juez,2PH533J1xudlfZ5GaHAdVX,Ozuna,Latin
3,Ella - Remix,7iHHlCbsFou5DZSW5HCPfs,Boza,Latin
4,Unfollow,6GsD646HrIDJJpzoY9T1Rt,Duki,Latin
...,...,...,...,...
264,Ay Vamos,6Ges5C2IE738iJh4HyQizQ,J Balvin,Latin
265,Love Not War (The Tampa Beat),4Lt6GXGzYsa1tgkv3nGSTm,Jason Derulo,Latin
266,BICHOTA,7vrJn5hDSXRmdXoR30KgF1,KAROL G,Latin
267,Hey DJ,1XgpK29CGGjZnxPYkiRbh4,CNCO,Latin


## Import playlists by genre
- I will import now a SPANISH playlist

In [136]:
## https://open.spotify.com/playlist/1oUJ31mriViDmfMliMXRRZ?si=c25b5ae7a9d546cb
## Gr Fausto

In [137]:
spanish_playlist = sp.user_playlist_tracks("Gr Fausto", "1oUJ31mriViDmfMliMXRRZ")

In [139]:
spanish_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [141]:
len(spanish_playlist['items'])

100

In [142]:
def get_playlist_tracks6(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [143]:
results8=get_playlist_tracks6("Gr Fausto", "1oUJ31mriViDmfMliMXRRZ")

In [144]:
len(results8)

597

In [145]:
results8[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [146]:
results8[0]['track']["uri"].rsplit(':',1)

['spotify:track', '3v1dCP3hk2djfWryqfp7sx']

In [147]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results8

In [148]:
for r in results8:
    song_URI.append(r["track"]["uri"])
    
for r in results8: 
    song_ID.append(r["track"]["id"])
    
for r in results8: 
    song_name.append(r["track"]["name"])
    
for r in results8: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [149]:
playlist_spanish_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})

In [150]:
playlist_spanish_info

,song_name,song_ID,song_artist
0,Caminando por la vida,3v1dCP3hk2djfWryqfp7sx,Melendi
1,Soldadito marinero,0eVborSuxUeSg0meWYd9dZ,Fito y Fitipaldis
2,Walter Palmeras,58UxBSr2y1b7gun2Vjt8dg,Taburete
3,Caminito A Motel,3VptrSkImJN0c5KCEBHE65,Taburete
4,Amos del Piano Bar,5wnIjet6vh2BaIX2gl4Mgz,Taburete
...,...,...,...
592,Cuando Brille el Sol (with Manuel Espana),2T0hFOkypPat3w4K5mc7fc,Melocos
593,Yo no te pido la luna,1hI1st8dvPmr1S55XDkavG,Sergio Dalma
594,A Mi Manera,5yY2UPVQ9EBKBXh2dGtU6U,Siempre Asi
595,Sopa fría,1zLJEnxuZBgPbwMrDMiZAf,M-Clan


### Add a new column for genre in playlist_spanish_info

In [151]:
playlist_spanish_info.insert(3, 'Genre', 'Spanish') 

In [152]:
playlist_spanish_info

,song_name,song_ID,song_artist,Genre
0,Caminando por la vida,3v1dCP3hk2djfWryqfp7sx,Melendi,Spanish
1,Soldadito marinero,0eVborSuxUeSg0meWYd9dZ,Fito y Fitipaldis,Spanish
2,Walter Palmeras,58UxBSr2y1b7gun2Vjt8dg,Taburete,Spanish
3,Caminito A Motel,3VptrSkImJN0c5KCEBHE65,Taburete,Spanish
4,Amos del Piano Bar,5wnIjet6vh2BaIX2gl4Mgz,Taburete,Spanish
...,...,...,...,...
592,Cuando Brille el Sol (with Manuel Espana),2T0hFOkypPat3w4K5mc7fc,Melocos,Spanish
593,Yo no te pido la luna,1hI1st8dvPmr1S55XDkavG,Sergio Dalma,Spanish
594,A Mi Manera,5yY2UPVQ9EBKBXh2dGtU6U,Siempre Asi,Spanish
595,Sopa fría,1zLJEnxuZBgPbwMrDMiZAf,M-Clan,Spanish


## Import playlists by genre
- I will import now a JAZZ playlist

In [153]:
## https://open.spotify.com/playlist/37i9dQZF1DX0SM0LYsmbMT?si=a455c71c0ea24707
## Spotify

In [154]:
jazz_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX0SM0LYsmbMT")

In [156]:
jazz_playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [158]:
len(jazz_playlist['items'])

100

In [159]:
def get_playlist_tracks7(user_id, playlist_id):
    results=sp.user_playlist_tracks(user_id, playlist_id)
    tracks=results['items']
    while results['next']:
        results= sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [160]:
results9=get_playlist_tracks7("Spotify", "37i9dQZF1DX0SM0LYsmbMT")

In [161]:
len(results9)

300

In [162]:
results9[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [163]:
results9[0]['track']["uri"].rsplit(':',1)

['spotify:track', '6p8cuVRnfBv4C6dhvYua5i']

In [164]:
song_name=[]
song_URI=[]
song_ID=[]
song_artist=[]
playlist=results9

In [165]:
for r in results9:
    song_URI.append(r["track"]["uri"])
    
for r in results9: 
    song_ID.append(r["track"]["id"])
    
for r in results9: 
    song_name.append(r["track"]["name"])
    
for r in results9: 
    song_artist.append(r["track"]["artists"][0]["name"])

In [166]:
playlist_jazz_info=pd.DataFrame({"song_name":song_name,"song_ID":song_ID,"song_artist":song_artist})

In [167]:
playlist_jazz_info

,song_name,song_ID,song_artist
0,Cayo,6p8cuVRnfBv4C6dhvYua5i,Chancla
1,Space Travels,188TF9y7eulHH0tKVZ6jxz,Mr. Käfer
2,Río Ocho,1cpPta2P0aGxUn6F6Up96n,Pachakuti
3,Bob's Law,4he3FjvSZ2owAuGGtb5Jt5,Pawcut
4,Dolce,2aXOl6fv48QvOFI2WxV4JZ,Lazlow
...,...,...,...
295,Yellow Napkins,72AJ5sfLfwtKHiQnUeB7rV,JAY VANILLA
296,Profesor,70A2dr0LJoFAmplOmYqdkn,Chancla
297,Midnight Stroll,00oSY6ZhNBI9uDf45e6VMw,CMJ
298,Good Life,5BPKNG6yHbj0Q2W6ksMbbH,Phlocalyst


### Add a new column for genre in playlist_spanish_info

In [168]:
playlist_jazz_info.insert(3, 'Genre', 'Jazz') 

In [169]:
playlist_jazz_info

,song_name,song_ID,song_artist,Genre
0,Cayo,6p8cuVRnfBv4C6dhvYua5i,Chancla,Jazz
1,Space Travels,188TF9y7eulHH0tKVZ6jxz,Mr. Käfer,Jazz
2,Río Ocho,1cpPta2P0aGxUn6F6Up96n,Pachakuti,Jazz
3,Bob's Law,4he3FjvSZ2owAuGGtb5Jt5,Pawcut,Jazz
4,Dolce,2aXOl6fv48QvOFI2WxV4JZ,Lazlow,Jazz
...,...,...,...,...
295,Yellow Napkins,72AJ5sfLfwtKHiQnUeB7rV,JAY VANILLA,Jazz
296,Profesor,70A2dr0LJoFAmplOmYqdkn,Chancla,Jazz
297,Midnight Stroll,00oSY6ZhNBI9uDf45e6VMw,CMJ,Jazz
298,Good Life,5BPKNG6yHbj0Q2W6ksMbbH,Phlocalyst,Jazz


## Joining all the dataframes into 1

In [170]:
frames=[playlist_indie_info, playlist_pop_info, playlist_rock_info, 
        playlist_tropical_info, playlist_hiphop_info
       ,playlist_latin_info,playlist_spanish_info,
       playlist_jazz_info]
big_df=pd.concat(frames, axis=0)

In [171]:
big_df

,song_name,song_ID,song_artist,Genre
0,Beggin',3Wrjm47oTz2sjIgck11l5e,Måneskin,Indie
1,The Less I Know The Better,4g3Ax56IslQkI6XVfYKVc5,Tame Impala,Indie
2,Mr. Brightside,3n3Ppam7vgaVa1iaRUc9Lp,The Killers,Indie
3,Boys Don't Cry,1QFh8OH1e78dGd3VyJZCAC,The Cure,Indie
4,A-Punk,5dKBaysNJtfpyNTRa5lqDb,Vampire Weekend,Indie
...,...,...,...,...
295,Yellow Napkins,72AJ5sfLfwtKHiQnUeB7rV,JAY VANILLA,Jazz
296,Profesor,70A2dr0LJoFAmplOmYqdkn,Chancla,Jazz
297,Midnight Stroll,00oSY6ZhNBI9uDf45e6VMw,CMJ,Jazz
298,Good Life,5BPKNG6yHbj0Q2W6ksMbbH,Phlocalyst,Jazz


In [172]:
## reseting the index so the rows appear correctly

big_df.reset_index(drop=True, inplace=True)

In [173]:
big_df

,song_name,song_ID,song_artist,Genre
0,Beggin',3Wrjm47oTz2sjIgck11l5e,Måneskin,Indie
1,The Less I Know The Better,4g3Ax56IslQkI6XVfYKVc5,Tame Impala,Indie
2,Mr. Brightside,3n3Ppam7vgaVa1iaRUc9Lp,The Killers,Indie
3,Boys Don't Cry,1QFh8OH1e78dGd3VyJZCAC,The Cure,Indie
4,A-Punk,5dKBaysNJtfpyNTRa5lqDb,Vampire Weekend,Indie
...,...,...,...,...
2431,Yellow Napkins,72AJ5sfLfwtKHiQnUeB7rV,JAY VANILLA,Jazz
2432,Profesor,70A2dr0LJoFAmplOmYqdkn,Chancla,Jazz
2433,Midnight Stroll,00oSY6ZhNBI9uDf45e6VMw,CMJ,Jazz
2434,Good Life,5BPKNG6yHbj0Q2W6ksMbbH,Phlocalyst,Jazz


In [174]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2436 entries, 0 to 2435
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   song_name    2436 non-null   object
 1   song_ID      2436 non-null   object
 2   song_artist  2436 non-null   object
 3   Genre        2436 non-null   object
dtypes: object(4)
memory usage: 76.2+ KB


# Audio features

In [175]:
from tqdm.notebook import tqdm

In [176]:
feats=[]

for song_ID in tqdm(range(len(big_df))):
    feats.extend(sp.audio_features(tracks=big_df['song_ID'][song_ID]))

  0%|          | 0/2436 [00:00<?, ?it/s]

In [178]:
feats[0].keys()

dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [179]:
feats_df=pd.DataFrame(feats)

In [180]:
feats_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.714,0.800,11,-4.808,0,0.0504,0.12700,0.0000,0.3590,0.589,134.002,audio_features,3Wrjm47oTz2sjIgck11l5e,spotify:track:3Wrjm47oTz2sjIgck11l5e,https://api.spotify.com/v1/tracks/3Wrjm47oTz2s...,https://api.spotify.com/v1/audio-analysis/3Wrj...,211560,4
1,0.640,0.755,1,-4.077,0,0.0287,0.01380,0.0208,0.1200,0.744,116.883,audio_features,4g3Ax56IslQkI6XVfYKVc5,spotify:track:4g3Ax56IslQkI6XVfYKVc5,https://api.spotify.com/v1/tracks/4g3Ax56IslQk...,https://api.spotify.com/v1/audio-analysis/4g3A...,216319,4
2,0.355,0.918,1,-4.360,1,0.0746,0.00119,0.0000,0.0971,0.240,148.114,audio_features,3n3Ppam7vgaVa1iaRUc9Lp,spotify:track:3n3Ppam7vgaVa1iaRUc9Lp,https://api.spotify.com/v1/tracks/3n3Ppam7vgaV...,https://api.spotify.com/v1/audio-analysis/3n3P...,222200,4
3,0.462,0.836,11,-5.766,0,0.0621,0.00517,0.0000,0.1800,0.671,168.766,audio_features,1QFh8OH1e78dGd3VyJZCAC,spotify:track:1QFh8OH1e78dGd3VyJZCAC,https://api.spotify.com/v1/tracks/1QFh8OH1e78d...,https://api.spotify.com/v1/audio-analysis/1QFh...,155973,4
4,0.551,0.821,2,-4.489,1,0.0539,0.01020,0.0438,0.1530,0.842,174.923,audio_features,5dKBaysNJtfpyNTRa5lqDb,spotify:track:5dKBaysNJtfpyNTRa5lqDb,https://api.spotify.com/v1/tracks/5dKBaysNJtfp...,https://api.spotify.com/v1/audio-analysis/5dKB...,137760,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2431,0.752,0.401,7,-6.086,0,0.5120,0.95000,0.0394,0.1270,0.926,81.718,audio_features,72AJ5sfLfwtKHiQnUeB7rV,spotify:track:72AJ5sfLfwtKHiQnUeB7rV,https://api.spotify.com/v1/tracks/72AJ5sfLfwtK...,https://api.spotify.com/v1/audio-analysis/72AJ...,99787,4
2432,0.831,0.423,10,-5.961,0,0.0616,0.21000,0.9330,0.0703,0.853,82.018,audio_features,70A2dr0LJoFAmplOmYqdkn,spotify:track:70A2dr0LJoFAmplOmYqdkn,https://api.spotify.com/v1/tracks/70A2dr0LJoFA...,https://api.spotify.com/v1/audio-analysis/70A2...,168705,4
2433,0.709,0.208,11,-18.080,0,0.1060,0.64500,0.7430,0.0907,0.428,173.993,audio_features,00oSY6ZhNBI9uDf45e6VMw,spotify:track:00oSY6ZhNBI9uDf45e6VMw,https://api.spotify.com/v1/tracks/00oSY6ZhNBI9...,https://api.spotify.com/v1/audio-analysis/00oS...,132414,4
2434,0.809,0.333,5,-11.839,0,0.0397,0.73500,0.9140,0.1110,0.610,89.946,audio_features,5BPKNG6yHbj0Q2W6ksMbbH,spotify:track:5BPKNG6yHbj0Q2W6ksMbbH,https://api.spotify.com/v1/tracks/5BPKNG6yHbj0...,https://api.spotify.com/v1/audio-analysis/5BPK...,140633,4


### Merge the two dataframes so you have the songs and artists with de features

In [181]:
big_df.merge(feats_df, left_on='song_ID', right_on='id')

,song_name,song_ID,song_artist,Genre,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Beggin',3Wrjm47oTz2sjIgck11l5e,Måneskin,Indie,0.714,0.800,11,-4.808,0,0.0504,...,0.3590,0.589,134.002,audio_features,3Wrjm47oTz2sjIgck11l5e,spotify:track:3Wrjm47oTz2sjIgck11l5e,https://api.spotify.com/v1/tracks/3Wrjm47oTz2s...,https://api.spotify.com/v1/audio-analysis/3Wrj...,211560,4
1,The Less I Know The Better,4g3Ax56IslQkI6XVfYKVc5,Tame Impala,Indie,0.640,0.755,1,-4.077,0,0.0287,...,0.1200,0.744,116.883,audio_features,4g3Ax56IslQkI6XVfYKVc5,spotify:track:4g3Ax56IslQkI6XVfYKVc5,https://api.spotify.com/v1/tracks/4g3Ax56IslQk...,https://api.spotify.com/v1/audio-analysis/4g3A...,216319,4
2,Mr. Brightside,3n3Ppam7vgaVa1iaRUc9Lp,The Killers,Indie,0.355,0.918,1,-4.360,1,0.0746,...,0.0971,0.240,148.114,audio_features,3n3Ppam7vgaVa1iaRUc9Lp,spotify:track:3n3Ppam7vgaVa1iaRUc9Lp,https://api.spotify.com/v1/tracks/3n3Ppam7vgaV...,https://api.spotify.com/v1/audio-analysis/3n3P...,222200,4
3,Boys Don't Cry,1QFh8OH1e78dGd3VyJZCAC,The Cure,Indie,0.462,0.836,11,-5.766,0,0.0621,...,0.1800,0.671,168.766,audio_features,1QFh8OH1e78dGd3VyJZCAC,spotify:track:1QFh8OH1e78dGd3VyJZCAC,https://api.spotify.com/v1/tracks/1QFh8OH1e78d...,https://api.spotify.com/v1/audio-analysis/1QFh...,155973,4
4,A-Punk,5dKBaysNJtfpyNTRa5lqDb,Vampire Weekend,Indie,0.551,0.821,2,-4.489,1,0.0539,...,0.1530,0.842,174.923,audio_features,5dKBaysNJtfpyNTRa5lqDb,spotify:track:5dKBaysNJtfpyNTRa5lqDb,https://api.spotify.com/v1/tracks/5dKBaysNJtfp...,https://api.spotify.com/v1/audio-analysis/5dKB...,137760,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463,Yellow Napkins,72AJ5sfLfwtKHiQnUeB7rV,JAY VANILLA,Jazz,0.752,0.401,7,-6.086,0,0.5120,...,0.1270,0.926,81.718,audio_features,72AJ5sfLfwtKHiQnUeB7rV,spotify:track:72AJ5sfLfwtKHiQnUeB7rV,https://api.spotify.com/v1/tracks/72AJ5sfLfwtK...,https://api.spotify.com/v1/audio-analysis/72AJ...,99787,4
2464,Profesor,70A2dr0LJoFAmplOmYqdkn,Chancla,Jazz,0.831,0.423,10,-5.961,0,0.0616,...,0.0703,0.853,82.018,audio_features,70A2dr0LJoFAmplOmYqdkn,spotify:track:70A2dr0LJoFAmplOmYqdkn,https://api.spotify.com/v1/tracks/70A2dr0LJoFA...,https://api.spotify.com/v1/audio-analysis/70A2...,168705,4
2465,Midnight Stroll,00oSY6ZhNBI9uDf45e6VMw,CMJ,Jazz,0.709,0.208,11,-18.080,0,0.1060,...,0.0907,0.428,173.993,audio_features,00oSY6ZhNBI9uDf45e6VMw,spotify:track:00oSY6ZhNBI9uDf45e6VMw,https://api.spotify.com/v1/tracks/00oSY6ZhNBI9...,https://api.spotify.com/v1/audio-analysis/00oS...,132414,4
2466,Good Life,5BPKNG6yHbj0Q2W6ksMbbH,Phlocalyst,Jazz,0.809,0.333,5,-11.839,0,0.0397,...,0.1110,0.610,89.946,audio_features,5BPKNG6yHbj0Q2W6ksMbbH,spotify:track:5BPKNG6yHbj0Q2W6ksMbbH,https://api.spotify.com/v1/tracks/5BPKNG6yHbj0...,https://api.spotify.com/v1/audio-analysis/5BPK...,140633,4


### I put into "Raw NBConvert" some lines to make the notebook easily to read